# PY OOP -  Ottimizzazione dei parametri

In [135]:
import pandas as pd
from scipy.integrate import odeint
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import random
from statistics import mean 
import itertools
import ga 
import math
path_file="https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv"
dataset=pd.read_csv(path_file,parse_dates=['data'])

# guariti,infetti,sani, morti 
dataset['asintomatici']= dataset['totale_casi'] * 0.44 
dataset['immuni'] = dataset['dimessi_guariti'] * 0.05  #verificare proporzione
data=dataset[['totale_positivi','dimessi_guariti','deceduti','asintomatici','immuni']]


def initializeParameters(minimo,massimo,n=9):
        return tuple([random.uniform(minimo,massimo) for i in range(0,n)])

def deriv(y,t,k1,k2,k3,k4,k5,k6,k7,k8,k9):  
    '''set di equazioni differenziali'''
    Sani,Infetti,Asintomatici,Morti,Guariti,Immuni = y 
    dSanidt =  -k1*Sani*Infetti -k2*Sani*Infetti-k3*Sani*Asintomatici- k4*Sani*Asintomatici
    dInfettidt = k1*Sani*Infetti+k4*Sani-k6*Infetti-k7*Infetti
    dAsintomaticidt =k2*Sani*Infetti+k3*Sani*Asintomatici-k5*Asintomatici
    dMortidt=k6*Infetti
    dGuaritidt = k5*Asintomatici+k7*Infetti-k8*Guariti-k9*Guariti
    dImmunidt = k9*Guariti
    return dSanidt,dInfettidt,dAsintomaticidt,dMortidt,dGuaritidt,dImmunidt  

def perturbazione(ls):
    sup = 0.2
    for i in range(len(ls)):
        if ls[i] < 0.5:
            ls[i]=random.uniform(ls[i],ls[i]+sup)
        else:
            ls[i]=random.uniform(0,ls[i]-sup)
    return(ls)

def diffusione(parameters,Sani = 6036000, Infetti=221,Guariti=1, Morti= 7,Immuni = 8,Asintomatici=10,orizzonte_temporale_in_gg=data.shape[0]):
        y0 =  [Sani,Infetti,Morti,Guariti,Immuni,Asintomatici]
        t = np.linspace(0,orizzonte_temporale_in_gg,orizzonte_temporale_in_gg)
        ret = odeint(deriv, y0, t,args=parameters)
        return(ret)

def fitness(previsti):
        df=pd.DataFrame(previsti,columns=['sani','totale_positivi','dimessi_guariti','deceduti','asintomatici','immuni'])
        df=df.drop(columns=['sani'])
        dataframe=((df-data)**2)
        lista=dataframe.values.tolist()
        merged = list(itertools.chain.from_iterable(lista))
        media=mean(merged)
        return media 

def localsearch(iniziale,iterazioni,minimo,massimo,param_only=True):
        #list(initializeParameters(minimo,massimo))
        k=iniziale.copy()
        errore= math.inf
        val='1'
        for c in range(iterazioni):
            valori=diffusione(tuple(k))
            stimaerrore=fitness(valori)
            if stimaerrore<errore:
                errore=stimaerrore
                val=valori 
                k=iniziale
            iniziale = list(initializeParameters(minimo,massimo))
        if param_only==True : 
            return k
        else : 
            return (k,val,errore)

def ils(iterazioni=10,minimo=0.00001,massimo=0.001):
        iniziale = list(initializeParameters(minimo,massimo))
        loc_search = localsearch(iniziale,iterazioni,minimo,massimo,param_only=False)
        k=loc_search[0]
        best_val = loc_search[1]
        best_errore = loc_search[2]
        #iter_errore = [best_errore]
        #iter_best_errore = [best_errore]
        val=0
        for c in range(iterazioni):
            val=best_val
            nuova_iniziale = perturbazione(k)
            loc_search = localsearch(nuova_iniziale,iterazioni,minimo,massimo,param_only=False)
            val = loc_search[1]
            if loc_search[2] < best_errore: # Se nuovo errore minore best si aggiorna il best
                best_errore = loc_search[2]
                best_val= loc_search[1]
                k=loc_search[0]
        # elif loc_search[0] == best_pers and loc_search[1] < best_num: # se uguale si tiene come best il numero più piccolo
                # best_num =loc_search[1]
            #iter_pers.append(loc_search[0])
        # iter_best_pers.append(best_pers)
    # results = {"errore":iter_pers, "persistenza_migliore":iter_best_pers, "numero_migliore":best_num,"ripetizioni":conta(best_num)}
        return(k,best_val,best_errore)  




In [136]:
ils(10,0.000001,0.01)

([0.3074680858551596,
  0.4257528225177376,
  0.5155031298604944,
  0.09313069034531285,
  0.32375067643925753,
  0.6562371839352676,
  0.22089946106960381,
  0.3106935088691911,
  0.4471615344759575],
 array([[ 6.03600000e+06,  2.21000000e+02,  7.00000000e+00,
          1.00000000e+00,  8.00000000e+00,  1.00000000e+01],
        [ 4.52251024e-29,  4.11624269e+05,  1.72410856e+06,
          1.81772037e+05,  1.03951318e+06,  1.04039717e+04],
        [-1.48256367e-28,  1.80627994e+05,  1.09590281e+06,
          2.61536686e+05,  1.25630003e+06,  3.03338564e+04],
        [-5.29819310e-28,  7.92627526e+04,  6.96593586e+05,
          2.96538821e+05,  1.14676343e+06,  5.07085780e+04],
        [ 2.74715899e-26,  3.47818952e+04,  4.42778886e+05,
          3.11898376e+05,  9.35872989e+05,  6.82006157e+04],
        [ 1.93126712e-26,  1.52629096e+04,  2.81445517e+05,
          3.18638420e+05,  7.19434430e+05,  8.20344851e+04],
        [-3.41253142e-27,  6.69763420e+03,  1.78896469e+05,
          3.